In [1]:
import pandas as pd
import json

# Load JSON file
with open('user-wallet-transactions.json', 'r') as f:
    data = json.load(f)

# If the JSON is a list of transactions
df = pd.DataFrame(data)




In [2]:
# Preview data
print(df.head())

                                    _id  \
0  {'$oid': '681d38fed63812d4655f571a'}   
1  {'$oid': '681aa70dd6df53021cc6f3c0'}   
2  {'$oid': '681d04c2d63812d4654c733e'}   
3  {'$oid': '681d133bd63812d46551b6ef'}   
4  {'$oid': '681899e4ba49fc91cf2f4454'}   

                                   userWallet  network protocol  \
0  0x00000000001accfa9cef68cf5371a23025b6d4b6  polygon  aave_v2   
1  0x000000000051d07a4fb3bd10121a343d85818da6  polygon  aave_v2   
2  0x000000000096026fb41fc39f9875d164bd82e2dc  polygon  aave_v2   
3  0x000000000096026fb41fc39f9875d164bd82e2dc  polygon  aave_v2   
4  0x0000000000e189dd664b9ab08a33c4839953852c  polygon  aave_v2   

                                              txHash  \
0  0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6...   
1  0xe6fc162c86b2928b0ba9b82bda672763665152b9de9d...   
2  0xe2d7eb815c89331a734ed6f204a06c385a1b39040baa...   
3  0x0d63a2eacd82b82f868db825ea7385e6bd8d046ee729...   
4  0x590eabb812c5006a6f4766f44e6e9d3ad0b5b563de69...   

 

In [3]:
# Flatten each document (handling nested $oid and $date)
for d in data:
    # Flatten MongoDB ID
    if '_id' in d and '$oid' in d['_id']:
        d['_id'] = d['_id']['$oid']

    # Flatten createdAt and updatedAt
    if 'createdAt' in d and '$date' in d['createdAt']:
        d['createdAt'] = d['createdAt']['$date']
    if 'updatedAt' in d and '$date' in d['updatedAt']:
        d['updatedAt'] = d['updatedAt']['$date']

# Convert to DataFrame
df = pd.DataFrame(data)

In [4]:
# Convert Unix timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

# Convert ISO date strings to datetime
df['createdAt'] = pd.to_datetime(df['createdAt'])
df['updatedAt'] = pd.to_datetime(df['updatedAt'])


In [5]:
# Normalize actionData dictionary
action_df = pd.json_normalize(df['actionData'])

# Merge into main DataFrame
df = pd.concat([df.drop(columns=['actionData']), action_df], axis=1)

# Convert amount from string to float (assuming it's in smallest units like wei)
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

# If in wei, convert to Ether (adjust if different token decimals)
df['amount_eth'] = df['amount'] / 1e18



In [6]:
# Optional cleanup
df = df.rename(columns={
    '_id': 'id',
    'userWallet': 'wallet',
    'txHash': 'tx_hash',
    'blockNumber': 'block_number',
    'type': 'action_type',
})

# Drop irrelevant fields if any
df.drop(columns=['__v', 'logId'], inplace=True, errors='ignore')


In [7]:
df.to_csv('cleaned_data.csv', index=False)
